In [ ]:
import sys
import json

general = {'step_size': '0.001', 'use_road_network': True, 'coef_day_to_year': '300', 'clustering_radius': '500'}
footpaths = {'max_length': '1000', 'speed': '2.8', 'n_ntlegs': '2'}

params = {
    'general': general,
    'footpaths': footpaths
    }

default = {'training_folder': '../../scenarios/clermont', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)

{'training_folder': '../../scenarios/clermont', 'params': {'catchment_radius': {'bus': '500', 'subway': '800', 'rail': '1000'}, 'footpaths': {'max_length': '1000', 'speed': '2.8', 'n_ntlegs': '2'}, 'use_road_network': True}}


In [2]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
import numpy as np
import random
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
from syspy.spatial.spatial import add_geometry_coordinates, nearest
from sklearn.neighbors import NearestNeighbors
from typing import Literal
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel

#num_cores = 1
print('numba threads',nb.config.NUMBA_NUM_THREADS)

on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
io_engine = 'pyogrio'

numba threads 20


In [3]:
def get_epsg(lat: float, lon: float) -> int:
    '''
    return EPSG in meter for a given (lat,lon)
    lat is north south 
    lon is est west
    '''
    return int(32700 - round((45 + lat) / 90, 0) * 100 + round((183 + lon) / 6, 0))

# Folders stucture and params

everything is on S3 (nothing on ECR) so no direct input folder. just scenarios/{scen}/inputs/

In [ ]:
use_road_network = argv['params']['general'].get('use_road_network')

In [5]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(input_folder, 'model/')


In [6]:
if 'footpaths' in argv['params'].keys():
    max_length = float(argv['params' ]['footpaths']['max_length'])
    speed = float(argv['params' ]['footpaths']['speed'])
    n_ntlegs = int(argv['params' ]['footpaths']['n_ntlegs'])
else:
    max_length = float(footpaths['max_length'])
    speed = float(footpaths['speed'])
    n_ntlegs = int(footpaths['n_ntlegs'])

# Inputs

In [7]:
links = gpd.read_file(pt_folder + 'links.geojson',engine=io_engine) 
nodes = gpd.read_file(pt_folder + 'nodes.geojson',engine=io_engine)
links = links.set_index('index')
nodes = nodes.set_index('index')

Skipping field road_link_list: unsupported OGR type: 5


In [8]:
rnodes_file = os.path.join(road_folder, 'road_nodes.geojson')
rnodes_file_provided = os.path.isfile(rnodes_file)
if rnodes_file_provided:
    rnodes = gpd.read_file(os.path.join(road_folder, 'road_nodes.geojson'), engine=io_engine)
    rnodes = rnodes.set_index('index')
    rlinks = gpd.read_file(os.path.join(road_folder, 'road_links.geojson'), engine=io_engine)
    rlinks = rlinks.set_index('index')
print('rnodes?',rnodes_file_provided)

rnodes? True


In [9]:
od_file = os.path.join(od_folder, 'od.geojson')
od_file_provided = os.path.isfile(od_file)

if od_file_provided:
    od_test = gpd.read_file(od_file, engine=io_engine)
    if 'index' not in od_test.columns:
        od_test.reset_index(names='index', inplace=True)
    if 'name' not in od_test.columns:
        od_test['name'] = od_test['index'].astype(str)
    od_test['name'] = od_test['name'].fillna(od_test['index'].astype(str))

    od_test = od_test[['name', 'origin', 'destination', 'volume', 'geometry']]
else:
    print('end of pathfinder')
    end_of_notebook

end_of_notebook here if OD_file not provided!!

In [10]:
od_test['volume'] = od_test['volume'].map(int).astype(float)
od_test = od_test.loc[od_test['volume'] >= 1]

In [11]:
od_test['geometry_o'] = od_test['geometry'].apply(lambda g: Point(g.coords[:][0]))
od_test['geometry_d'] = od_test['geometry'].apply(lambda g: Point(g.coords[:][1]))

if 'origin' not in od_test.columns:
    od_test['origin'] = od_test['index'].astype(str) + '_o' 
od_test['origin'] = od_test['origin'].astype(str)

if 'destination' not in od_test.columns:
    od_test['destination'] = od_test['index'].astype(str) + '_d' 
od_test['destination'] = od_test['destination'].astype(str)

zones = od_test.copy()
zones_d = od_test.copy()
zones['geometry'] = zones['geometry_o']
zones_d['geometry'] = zones_d['geometry_d']
zones['index'] = zones['origin'].astype(str)
zones_d['index'] = zones_d['destination'].astype(str)
zones = pd.concat([zones[['index','geometry']],zones_d[['index','geometry']]]).drop_duplicates()
zones = zones.set_index('index')

od_set = set(zip(od_test['origin'], od_test['destination']))

# preparation

In [12]:
def add_col(links,col='length'):
    if col not in links.columns:
        return True
    elif any(links[col].isnull()):
        return True
    else:
        return False

In [13]:
if add_col(links,'length'):
    print('add length')
    centroid = [*LineString(nodes.centroid.values).centroid.coords][0]
    crs = get_epsg(centroid[1],centroid[0])
    print('create length from geometry')
    links['length'] = links.to_crs(crs).length
    
if add_col(links,'speed'):
    print('add speed')
    links['speed'] = links['length']/links['time'] * 3.6

# Walkmodel

In [14]:
sm = stepmodel.StepModel(epsg=4326)
sm.links = links
sm.nodes = nodes
sm.zones = zones

if use_road_network:
    sm.road_links = rlinks
    sm.road_links['walk_time'] = sm.road_links['length'] / (3000/3600)
    
    sm.road_nodes = rnodes

In [15]:
# We remove highway for the walkmodel
#walk_type = ['Artere','Collectrices','Locale','Nationale','Regionale']
#walkmodel.road_links = walkmodel.road_links[~walkmodel.road_links['highway'].isin(walk_type)]
#walkmodel.road_nodes = walkmodel.road_nodes.loc[list(np.unique(walkmodel.road_links[['a','b']].values.flatten()))]
#walkmodel.preparation_clusterize_nodes(distance_threshold=0.000002)
# Footpath between PT nodes
sm.preparation_footpaths(speed=speed, max_length=max_length, clusters_distance=0.00001)

In [16]:
# Access footpaths (zone_to_road and road_to_transit)
sm.preparation_ntlegs(
    short_leg_speed=speed,
    long_leg_speed=speed, # tout le monde marche
    threshold=1000,
    zone_to_transit=(not use_road_network),
    zone_to_road=use_road_network,
    road_to_transit=use_road_network,
    n_ntlegs=n_ntlegs,
    max_ntleg_length=200000,
)

In [17]:
# Copy connectors to the model
sm.footpaths = sm.footpaths.drop(columns='voronoi')
#sm.footpaths = walkmodel.footpaths.drop(columns='voronoi').copy()
#sm.zone_to_transit = walkmodel.zone_to_transit.copy()

In [18]:
sm.footpaths['speed'] = sm.footpaths['length']/sm.footpaths['time'] * 3.6

# utils

# pathfinder

test test test

In [19]:
sm.step_pt_pathfinder(
    broken_routes=False,
    broken_modes=False,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    speedup=True,
    walk_on_road=use_road_network,
    path_analysis=False,
    od_set=od_set
)

start publicpathfinder
897 sources 897 targets direct search
path_analysis


In [20]:
if use_road_network:
    sm.zone_to_road.reset_index(drop=True, inplace=True)
    sm.zone_to_road.reset_index(names='index', inplace=True)
    sm.zone_to_road['index'] = sm.zone_to_road['index'].apply(lambda x: 'ztr_' + str(x))
    sm.zone_to_road.set_index('index', inplace=True)

    sm.road_to_transit.reset_index(drop=True, inplace=True)
    sm.road_to_transit.reset_index(names='index', inplace=True)
    sm.road_to_transit['index'] = sm.road_to_transit['index'].apply(lambda x: 'rtt_' + str(x))
    sm.road_to_transit.set_index('index', inplace=True)

else:
    sm.zone_to_transit.reset_index(drop=True, inplace=True)
    sm.zone_to_transit.reset_index(names='index', inplace=True)
    sm.zone_to_transit['index'] = sm.zone_to_transit['index'].apply(lambda x: 'ztt_' + str(x))
    sm.zone_to_transit.set_index('index', inplace=True)

In [21]:
def extract_rlink_path(path, rlinks):
    links_path_set = {f"{path[i]}__{path[i+1]}" for i in range(len(path) - 1)}
    res = rlinks.loc[rlinks['ab_id'].isin(links_path_set), 'index'].values.tolist()
    return res

In [22]:
if use_road_network:
    rlinks = sm.road_links[['a', 'b', 'geometry']].copy().reset_index(names='index')
    rlinks['ab_id'] = rlinks['a'] + '__' + rlinks['b']

    paths = sm.pt_los['path'].values.tolist()
    for path in paths:
        g = extract_rlink_path(path, rlinks)

In [23]:
sm.analysis_pt_los(walk_on_road=use_road_network)

path_analysis: 100%|██████████| 45317/45317 [00:05<00:00, 8916.77it/s] 


# create path

In [24]:
od_links = od_test.merge(sm.pt_los[['origin','destination','gtime','link_path']],on=['origin','destination'])
od_links = od_links.drop(columns = ['geometry','geometry_o','geometry_d','origin','destination'])
od_links = od_links.explode('link_path')
# may add route_width if quenedi can handle it one day.
od_links = od_links.merge(sm.links[['route_color','geometry','time','speed']],left_on='link_path',right_index=True)
od_links = od_links.drop(columns='link_path')
od_links['route_color'] = '#838383'

In [ ]:
od_ntlegs = od_test.merge(sm.pt_los[['origin','destination','gtime','ntlegs']],on=['origin','destination'])
od_ntlegs = od_ntlegs.drop(columns = ['geometry','geometry_o','geometry_d','origin','destination'])
od_ntlegs = od_ntlegs.explode('ntlegs')

#od_ntlegs
if use_road_network:
    ntlegs_dict1 = sm.zone_to_road.reset_index().set_index(['a','b'])['index'].to_dict()
    ntlegs_dict2 = sm.road_to_transit.reset_index().set_index(['a','b'])['index'].to_dict()
    od_ntlegs['ntlegs'] = od_ntlegs['ntlegs'].apply(lambda x: ntlegs_dict1.get(x, ntlegs_dict2.get(x, None)))

    od_ntlegs = od_ntlegs.merge(pd.concat([sm.zone_to_road[['geometry','time','speed']], sm.road_to_transit[['geometry','time','speed']]])
                                , left_on='ntlegs'
                                , right_index=True
                                , how='left')

else:
    ntlegs_dict = sm.zone_to_transit.reset_index().set_index(['a','b'])['index'].to_dict()
    od_ntlegs['ntlegs'] = od_ntlegs['ntlegs'].apply(lambda x: ntlegs_dict.get(x, ntlegs_dict.get(x, None)))

    od_ntlegs = od_ntlegs.merge(sm.zone_to_transit[['geometry','time','speed']], left_on='ntlegs',right_index=True)


od_ntlegs = od_ntlegs.drop(columns='ntlegs')
od_ntlegs['route_color']='4B4B4B'

In [26]:
od_footpaths = od_test.merge(sm.pt_los[['origin','destination','gtime','footpaths']],on=['origin','destination'])
od_footpaths = od_footpaths.drop(columns = ['geometry','geometry_o','geometry_d','origin','destination'])
od_footpaths = od_footpaths.explode('footpaths')

footpaths_dict = sm.footpaths.reset_index().set_index(['a','b'])['index'].to_dict()
od_footpaths['footpaths'] = od_footpaths['footpaths'].apply(footpaths_dict.get)

od_footpaths = od_footpaths.merge(sm.footpaths[['geometry','time','speed']],left_on='footpaths',right_index=True)
od_footpaths = od_footpaths.drop(columns='footpaths')
od_footpaths['route_color'] = '#838383'

In [27]:
od_route = pd.concat([od_links,od_footpaths,od_ntlegs],axis=0)

In [28]:
od_route['route_color'] = od_route['route_color'].fillna('#838383')

In [29]:
od_route = od_route.rename(columns={'name':'od_name'})
od_route.reset_index(drop=True)
od_route.index.name='index'

In [30]:
od_route = gpd.GeoDataFrame(od_route,crs=4326)
#od_route.to_file(output_folder+'od_route.geojson',driver='GeoJSON',engine=io_engine)

# Formation :
Exporter des metrics (du pt_los) sur les OD trouvés en format csv <br>
 (ex: time, mode, etc). garder la colonne "name" s.v.p



In [31]:
sm.analysis_pt_route_type(['rail','subway','bus'])
pt_los = od_test[['origin','destination','name']].merge(sm.pt_los,on=['origin','destination'])

In [32]:
print('pathfinder successfull')

pathfinder successfull


# test